<a href="https://colab.research.google.com/github/tgandor/urban_oculus/blob/master/colab/Validate_multiple_JPEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%time
import os
if not os.path.isdir('urban_oculus'):
  !git clone https://github.com/tgandor/urban_oculus
# first, this doesn't kill runtime
%run urban_oculus/snippets/get_coco_val2017_nounzip.ipynb
# this may kill runtime
%run urban_oculus/snippets/get_detectron2.ipynb

Cloning into 'urban_oculus'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 72 (delta 35), reused 33 (delta 10), pack-reused 0
Unpacking objects: 100% (72/72), done.
Archive:  annotations_trainval2017.zip
  inflating: datasets/coco/annotations/instances_train2017.json  
  inflating: datasets/coco/annotations/instances_val2017.json  
  inflating: datasets/coco/annotations/captions_train2017.json  
  inflating: datasets/coco/annotations/captions_val2017.json  
  inflating: datasets/coco/annotations/person_keypoints_train2017.json  
  inflating: datasets/coco/annotations/person_keypoints_val2017.json  
--2020-02-20 16:06:30--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.142.52
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.142.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Le

In [0]:
import os
import shutil

def filesizes(glob_expr):
  import glob
  return {
      os.path.basename(f): os.path.getsize(f)
      for f in glob.glob(glob_expr)
  }


def validate_coco2017(quality, output_dir=None):
  def save(path):
    if output_dir:
      os.makedirs(output_dir, exist_ok=True)
      shutil.copy(path, output_dir)
      print(f'Saving {path} to {output_dir}')

  import json
  import time
  print('unpacking')
  !unzip -o val2017.zip -d datasets/coco 2>&1 | tqdm --total 5000 >/dev/null
  !du -h datasets/coco/val2017
  with open(f'filesizes_original.json', 'w') as f:
    f.write(json.dumps({
        'quality': 'original',
        'filesizes': filesizes('datasets/coco/val2017/*.jpg'),
        'elapsed': 0,
    }))
  save('filesizes_original.json')  

  print(f'degrading quality to {quality}')
  start = time.time()
  !mogrify -verbose -quality {quality} datasets/coco/val2017/*.jpg 2>&1 | tqdm --total 10000 >/dev/null
  elapsed = time.time() - start
  !du -h datasets/coco/val2017

  import detectron2.model_zoo
  from detectron2.config import get_cfg
  from detectron2.data import build_detection_test_loader
  from detectron2.evaluation import COCOEvaluator, inference_on_dataset
  from detectron2.engine import DefaultPredictor
  cfg = get_cfg()
  cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
  cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
  predictor = DefaultPredictor(cfg)
  data_loader = build_detection_test_loader(cfg, 'coco_2017_val')
  evaluator = COCOEvaluator('coco_2017_val', cfg, False)
  start = time.time()
  results = inference_on_dataset(predictor.model, data_loader, evaluator)
  inference_time = time.time() - start

  result_file = f'coco_val2017_rcnn_3x_q_{quality}.json'
  with open(result_file, 'w') as f:
    f.write(json.dumps({
        'quality': quality,
        'bbox': results['bbox'],
        'elapsed': inference_time,
    }))
  save(result_file)
  
  sizes_file = f'filesizes_{quality}.json'
  with open(sizes_file, 'w') as f:
    f.write(json.dumps({
        'quality': quality,
        'filesizes': filesizes('datasets/coco/val2017/*.jpg'),
        'elapsed': elapsed,
    }))
  save(sizes_file)

  # running locally
  from IPython.core.display import display, HTML
  display(HTML(f'Results: <a href="{result_file}" target="_blank">{result_file}</a> (not on Colab)'))

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# !ls /content/gdrive/My\ Drive/results

In [0]:
%%time
import time
output_dir = None
if os.path.exists('/content/gdrive'):
  output_dir = f'/content/gdrive/My Drive/results/{time.strftime("%Y%m%d_%H%M%S")}_results'
  print('Using output dir:', output_dir)

# this should go into some snippet:
try:
  from tqdm.notebook import tqdm
except ImportError:
  from tqdm import tqdm_notebook as tqdm

for q in tqdm(range(1, 101, 16)):
  validate_coco2017(q, output_dir)

In [0]:
import time
now = time.strftime('%Y%m%d_%H%M%S')
all_jsons = f"{now}_results.tar.gz"
# adding 'surrounding' folder around JSONs (avoiding tarbomb)
!tar --xform 's+^+{now}_results/+' -czf {all_jsons} *.json
!ls -l

In [0]:
import os
if os.path.exists('/content/gdrive'):
  !mkdir -p /content/gdrive/My\ Drive/results
  !cp -v {all_jsons} /content/gdrive/My\ Drive/results/
  !ls /content/gdrive/My\ Drive/results/

In [0]:
try:
  from google.colab import files
  files.download(all_jsons)
except ImportError:
  # running locally
  from IPython.core.display import display, HTML
  display(HTML(f'Results archive: <a href="{all_jsons}" target="_blank">{all_jsons}</a>'))

In [0]:
import torch

print('Device count:', torch.cuda.device_count())

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3, 1), 'GB')


In [0]:
!head -n 11 /proc/cpuinfo

In [1]:
!ls                                                                                        

sample_data
